# `radial_leaf` function

**A function `radial_leaf` that takes inputs of:**

1. leaf trace coordinates
2. landmark coordinates
3. number of points to interpolate for the trace
4. number of landmarks (interpolated points within each leaflet)
5. polynomial degree to model each landmark across leaflets in polar coordinates
6. frames (or x values) to model across the leaf, apportioned to intervals between leaflets by rounding

**The function returns:**

1. overall modeled x and y values. for both x and y values, this is a double list of lists structured first by frame, and the for each frame, modeled values for each landmark. models are using polar coordinates
2. interpolated trace lists for plotting
3. landmark lists that have been aligned to the trace (for plotting)

**IMPORTANT NOTE:** for now, only the right side of leaves is considered. The `np.arctan` function returns phi values only between +/- pi/2. There are also with problems in quadrant 4, in addition to 2 and 3, as well. This is a bug to figure out in the future, but is circumvented here by only considering the right side of leaves for the radial modeling step.

In [1]:
import numpy as np # for text file data
from PIL import Image # for displaying images
from scipy.interpolate import interp1d # for interpolating points
import matplotlib.pyplot as plt # for plotting
%matplotlib inline

# first define a function to return equally spaced, interpolated points

def interpolation(x, y, number): 

    distance = np.cumsum(np.sqrt( np.ediff1d(x, to_begin=0)**2 + np.ediff1d(y, to_begin=0)**2 ))
    distance = distance/distance[-1]

    fx, fy = interp1d( distance, x ), interp1d( distance, y )

    alpha = np.linspace(0, 1, number)
    x_regular, y_regular = fx(alpha), fy(alpha)
    
    return x_regular, y_regular

# define functions for converting between cartesian and polar coordinates

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

########################################
########################################
########################################
########################################

# now define the model_leaf function

def radial_leaf(trace_file, landmarks_file, num_points, num_land, deg, frames):

########################################
########################################
########################################
########################################

    trace_xvals, trace_yvals = interpolation(trace_file[:,0],trace_file[:,1],num_points)

    #############################
    # find interpolated trace points that correspond to the landmarks
    #############################

    # find the corresponding trace coordinates for each landmark
    # record new landmark coord values
    # record landmark index values for trace values

    land_xvals = [] # list to store new landmark x vals
    land_yvals = [] # list to store new landmark y vals
    land_indices = [] # list to store index values

    for i in range(landmarks_file.shape[0]):

        landx = landmarks_file[i,0] # select current landmark
        landy = landmarks_file[i,1] 

        distances = [] # list to store distances of current landmark with each trace coord

        for j in range(len(trace_xvals)): # for each trace coord

            tracex = trace_xvals[j]
            tracey = trace_yvals[j]

            d = np.sqrt( (landx-tracex)**2 + (landy-tracey)**2 ) # find distance to each landmark

            distances.append(d) # append to distances

        min_val = np.min(distances) # find min distance value
        min_ind = distances.index(min_val) # find min index value among trace coords

        land_xvals.append(trace_xvals[min_ind]) # append new landmark coord and index vals
        land_yvals.append(trace_yvals[min_ind])
        land_indices.append(min_ind)

    #############################
    # create same number of interpolated landmarks for each lobe, using beginning, tip, and end as true landmarks
    #############################

    num_lobes = int((len(land_indices)-1)/2) # get number of lobes in the leaf

    lobe_xvals = []
    lobe_yvals = []

    for i in range(num_lobes): # for the number of lobes

        begin = i*2 # current lobe beginning index
        tip = i*2+1 # current lobe tip index
        end = i*2+2 # current lobe end index

        first_half_x = trace_xvals[land_indices[begin]:land_indices[tip]] # isolate first half of traced lobe
        first_half_y = trace_yvals[land_indices[begin]:land_indices[tip]]
        second_half_x = trace_xvals[land_indices[tip]:land_indices[end]]  # isolate second half of traced lobe
        second_half_y = trace_yvals[land_indices[tip]:land_indices[end]]

        first_landx, first_landy = interpolation(first_half_x, first_half_y, int(num_land/2)) # interpolate half number of landmarks to first half
        second_landx, second_landy = interpolation(second_half_x, second_half_y, int(num_land/2)) # interpolate half number of landmarks to second half

        lobex = list(first_landx) + list(second_landx) # combine xvals of first and second half of lobe
        lobey = list(first_landy) + list(second_landy) # combine yvals of first and second half of lobe

        lobe_xvals.append(lobex) # append a list with the interpolated coordinate values for each lobe
        lobe_yvals.append(lobey)

    lobex_arr = np.array(lobe_xvals) # turn list of lists of lobe values into arrays
    lobey_arr = np.array(lobe_yvals)

    #############################
    # for frame number, apportion frames approximately equally between intervals by rounding
    #############################

    num_lobes = int((len(land_indices)-1)/2) # get number of lobes in the leaf

    inv_num = np.linspace(0,frames,num_lobes) # calculate interval number (between lobe spaces)

    round_inv = [int(round(item, 0)) for item in inv_num] # rounds the above list to number of frames for each interval

    intervals = [] # a list of lists, for each interval, a list of the frames it is apportioned

    for r in range(len(round_inv)-1):

        inv_begin = round_inv[r]
        inv_end = round_inv[r+1]

        interval = []

        for n in range(inv_begin, inv_end):

            interval.append(n)

        intervals.append(interval)

    #############################
    # create double list of lists, for each frame, a list of all modeled values
    #############################

    #################

    # First, translate coords to origin then convert to polar coords

    origin = landmarks_file[0] # calculate origin from first coords
    trans_lobesx = lobex_arr-origin[0] # translate lobe xvals to origin
    trans_lobesy = lobey_arr-origin[1] # translate lobe yvals to origin
    rho, phi = cart2pol(trans_lobesx, trans_lobesy) # convert x and y arrays to rho and phi

    #################

    overall_phis = [] # a double list of lists, for each frame, a list of phi values
    overall_rhos = [] # a double list of lists, for each frame, a list of rho values

    for f in range(frames): # for each frame

        curr_frame_phis = [] # a list of modeled values for each landmark for the current frame
        curr_frame_rhos = []

        for l in range(num_land):

            # for current landmark l, code below recalculates and apportions phis across leaf
            # this code is repetitive, but puts frame number before calculating modeled values
            all_phis = []
            for k in range(len(intervals)):
                curr_inv = intervals[k]
                inv_len = len(curr_inv)
                curr_phis = np.linspace(phi[k,l],phi[k+1,l], inv_len)
                all_phis.append(curr_phis)
            flat_all_phis = []
            for i in range(len(all_phis)):
                curr_list = all_phis[i]
                for j in range(len(curr_list)):
                    flat_all_phis.append(curr_list[j])

            # isolate the current xval for the current frame, for the current landmark
            curr_phi = flat_all_phis[f]

            # recalculates polynomial function for the current landmark l
            coef = np.polyfit(phi[:,l],rho[:,l],deg) # fit a polynomial for the current landmark
            polyfunc = np.poly1d(coef) # create polynomial function

            # for the current frame and landmark, find modeled value for the calculated xval
            modeled_val = polyfunc(curr_phi) 

            curr_frame_phis.append(curr_phi) # append current phi
            curr_frame_rhos.append(modeled_val) # append current modeled val

        overall_phis.append(curr_frame_phis) # append list of xvals for current frame
        overall_rhos.append(curr_frame_rhos) # append list of yvals for current frame

    #############################
    # convert polar back to cartesian and translate back to original location
    #############################

    trans_overall_xvals, trans_overall_yvals = pol2cart(overall_rhos, overall_phis)

    overall_xvals = trans_overall_xvals + origin[0] # translate lobe xvals back to original location
    overall_yvals = trans_overall_yvals + origin[1] # translate lobe yvals back to original location
    
    return overall_xvals, overall_yvals, trace_xvals, trace_yvals, land_xvals, land_yvals



# data for modeling different *Cannabis* leaves

various leaf outlines of *sativa*, *indica*, and hybrid varieties

In [47]:
# set the parameters

num_points = 200 # number of trace points to interpolate
num_land = 20 # number of landmarks for each lobe, make it even number
deg = 2 # degrees of fitted polynomial
frames = 50 # number of frames across the leaf to model

# load files

### A ###

Atrace = np.loadtxt("./data/Atrace.txt" ) # polyline trace
Aland = np.loadtxt("./data/Aland.txt" ) # leaflet landmarks

### B ###

Btrace = np.loadtxt("./data/Btrace.txt" ) # polyline trace
Bland = np.loadtxt("./data/Bland.txt" ) # leaflet landmarks

### C ###

Ctrace = np.loadtxt("./data/Ctrace.txt" ) # polyline trace
Cland = np.loadtxt("./data/Cland.txt" ) # leaflet landmarks



# use function radial_leaf to retrieve data

overAX, overAY, traceAX, traceAY, landAX, landAY = radial_leaf(Atrace, Aland, num_points, num_land, deg, frames)

overBX, overBY, traceBX, traceBY, landBX, landBY = radial_leaf(Btrace, Bland, num_points, num_land, deg, frames)

overCX, overCY, traceCX, traceCY, landCX, landCY = radial_leaf(Ctrace, Cland, num_points, num_land, deg, frames)




# Visualize data

In [52]:
######################
# specify data to plot

outlineAX = traceAX
outlineAY = traceAY
ptAX = landAX
ptAY = landAY
leafAX = overAX
leafAY = overAY

outlineBX = traceBX
outlineBY = traceBY
ptBX = landBX
ptBY = landBY
leafBX = overBX
leafBY = overBY

outlineCX = traceCX
outlineCY = traceCY
ptCX = landCX
ptCY = landCY
leafCX = overCX
leafCY = overCY

######################

photo_file = Image.open("./cannabis_leaves.jpg", 'r')

outline_c = "green"
outline_lw = 2

land_c = "dodgerblue"
land_s = 80

point_c = "magenta" # modeled point parameters
point_lw = 2

highlight_c = "orange" # highlight point parameters
highlight_s = 80

for f in range(frames):

    fig = plt.figure(figsize = (12*1.2,6*1.2), facecolor="black")

    plt.imshow(photo_file, cmap="gist_gray")
    
    ### A ###

    plt.plot(outlineAX,outlineAY, c=outline_c, lw=outline_lw)
    plt.scatter(ptAX,ptAY, c=land_c, s=land_s)

    plt.plot(leafAX[f], leafAY[f], c=point_c, lw=point_lw)
    plt.scatter(leafAX[f][0], leafAY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(leafAX[f][10], leafAY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(leafAX[f][19],  leafAY[f][19], c=highlight_c, s=highlight_s)
    
    ### B ###
    
    plt.plot(outlineBX,outlineBY, c=outline_c, lw=outline_lw)
    plt.scatter(ptBX,ptBY, c=land_c, s=land_s)
    
    plt.plot(leafBX[f], leafBY[f], c=point_c, lw=point_lw)
    plt.scatter(leafBX[f][0], leafBY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(leafBX[f][10], leafBY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(leafBX[f][19],  leafBY[f][19], c=highlight_c, s=highlight_s)
    
    ### C ###
    
    plt.plot(outlineCX,outlineCY, c=outline_c, lw=outline_lw)
    plt.scatter(ptCX,ptCY, c=land_c, s=land_s)
    
    plt.plot(leafCX[f], leafCY[f], c=point_c, lw=point_lw)
    plt.scatter(leafCX[f][0], leafCY[f][0], c=highlight_c, s=highlight_s)
    plt.scatter(leafCX[f][10], leafCY[f][10], c=highlight_c, s=highlight_s)
    plt.scatter(leafCX[f][19],  leafCY[f][19], c=highlight_c, s=highlight_s)
    
    
    #############
    
    plt.axis("off")
    #plt.ylim(670,-10)
    #plt.xlim(-10,1100)
    
    filename = "./plots/frame"+str(f)+".jpg"
        
    plt.savefig(filename, facecolor=fig.get_facecolor()) 
    plt.close()